In [1]:
# Tratamiento de datos
# ===============================================================================
import pandas as pd
import numpy as np
import os
import datetime
import random
import string
import locale
# Ajusta la configuración locale al formato de tu país
locale.setlocale(locale.LC_ALL, 'es_ES')

pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Importar clientes
# ===============================================================================
import os 
from dotenv import load_dotenv
load_dotenv()

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Guardar modelo
# ==============================================================================
import pickle

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Cargamos las credenciales
# ==============================================================================

publisher = os.getenv('publisher')
client_list = os.getenv("client_id")
client_list = client_list.split(',')
media_type = os.getenv('media_type')
industria = os.getenv('industria')

In [3]:
# Cargamos los ficheros
# ===============================================================================

data_dir = "../data/"
file_prefix = "2022-"
fields = ['date','brand_id_name','transaction_type_name','format_name',
                         'device_name', media_type,'publisher_id_name','revenue_resold','impressions_resold', industria]

# Obtenemos una lista de todos los ficheros del directorio  ../data/ con el prefijo 2022-
files = [f for f in os.listdir(data_dir) if f.startswith(file_prefix)]

# Lee los datos con las columnas especificadas en los campos, para cada fichero y los almacena en una lista
dataframes = []
for file in files:
    df = pd.read_csv(os.path.join(data_dir, file), sep=";",usecols=fields)
    dataframes.append(df)

# Concatena la lista de dataframes en un único dataframe y dropea los duplicados
df2022 = pd.concat(dataframes).drop_duplicates()

# Renombramos la columna media_type e industria
df2022 = df2022.rename(columns={media_type: 'media_type', industria: 'industria'})

In [4]:
# Comprobamos si hay duplicados, ya que representan el 16,36% del total de los datos procedemos a dropearlos en el código de llamada al csv
# ===============================================================================

# dup_percent = (df2022.duplicated().sum() / df2022.shape[0]) * 100
# print(f"The percentage of duplicated rows in df2022 is: {dup_percent:.2f}%")

In [5]:
# Hacemos una copia del df para hacer nuestros ajustes
# ===============================================================================

df2022_ = df2022.copy()

In [6]:
# Juntamos las categorías del transaction name para no desvirtuar la parte de Open
# ===============================================================================

df2022_['transaction_type_name'] = df2022_['transaction_type_name'].replace(['Private auction deal','Direct remnant','Curated marketplace'], 'Open auction')

In [7]:
# Agrupamos las columnas y generamos un nuevo df teniendo en cuenta los nan values y ordenando de mayor a menor
# ===============================================================================

df2022_gp = df2022_.groupby(['date', 'transaction_type_name', 'brand_id_name', 'publisher_id_name',
                            'format_name', 'device_name', 'media_type', 'industria'],
                         dropna=False).agg({'revenue_resold': 'sum','impressions_resold' :'sum'}).sort_values('revenue_resold', ascending=False).reset_index()

In [8]:
# Comprobamso que no se pierde información con el groupby
# ===============================================================================

df2022_gp['revenue_resold'].sum()-df2022['revenue_resold'].sum()

1.210719347000122e-08

In [9]:
# Comprobamos si hay nulos
# ===============================================================================

(df2022.isnull().sum() / len(df2022) * 100).round(2)

date                     0.00
transaction_type_name    0.00
brand_id_name            0.04
publisher_id_name        0.00
format_name              0.00
device_name              4.73
media_type               0.00
industria                0.04
revenue_resold           0.00
impressions_resold       0.00
dtype: float64

In [10]:
# Calculamos el CPM
# ===============================================================================

df2022_gp['cpm'] = (df2022_gp['revenue_resold']/df2022_gp['impressions_resold']*1000).round(2)

In [11]:
# Hacemos una copia de nuestro df para quedarnos con la info que nos interesa para el estudio de nuestros datos
# ===============================================================================

df = df2022_gp.copy()

# Quitamos las campañas de venta directa porque no interesan para el estudio
df = df[df['transaction_type_name']!='Direct guaranteed']

# Filtramos por nuestro publisher
df = df[df['publisher_id_name']==publisher]

# El device_name Connected TV no nos sirve para el estudio y hay que quitarlo
df = df[df['device_name']!='Connected TV']

# Nos quedamos solo con los media type del filtro ya que son los predominantes
df = df[df['media_type'].isin(['Web', 'Video', 'App'])]

# Tenemos que quitar los formatos menos relevantes para no desvirtuar los datos
df = df[df['format_name'].isin(['300x600 (Half Page)','320x50 (Mobile Banner)',
                                            '300x250 (Medium Rectangle)','Video','1800x1000 (Wallpaper/Skin)',
                                            '970x250 (Billboard)','320x100 (Mobile Double Size Banner)',
                                            'Interstitial','728x90 (Leaderboard - Mobile Banner Landscape)',
                                            'Animated Skin VPAID','300x300','Native','160x600 (Wide Skyscraper)'])]

# Creamos un diccionario con los formatos que queremos reemplazar
Format_Names = {
 '300x600 (Half Page)':'Doble Robapaginas',
 '1800x1000 (Wallpaper/Skin)':'Skin',
 '320x50 (Mobile Banner)': 'Mobile Banner',
 '300x250 (Medium Rectangle)':'Medium Rectangle', 
 'Video': 'Video',
 '970x250 (Billboard)':'Billboard',
 '320x100 (Mobile Double Size Banner)':'Doble BabyBanner',
 'Interstitial':'Intertitial', 
 '728x90 (Leaderboard - Mobile Banner Landscape)':'Mobile Banner',
 'Animated Skin VPAID': 'Animated Skin',
 'Native':'Native',
 '160x600 (Wide Skyscraper)':'Wide Skyscraper',
 '300x300':'Robapaginas'
 }

# Reemplazamos los números con el nombre del diccionario en la columna
df['format_name'] = df['format_name'].replace(Format_Names)

# Filtramos por los clientes con más inversión en el 2022
df = df[df['brand_id_name'].isin(client_list)]

# Dropeamos la columna publisher_id_name ya que no nos sirve ahora
df = df.drop('publisher_id_name', axis=1)

In [12]:
# Cargamos los resultados de los datos de GA y de Weather en el periodo de 2022
# ===============================================================================

df_GA = pd.read_csv('../data/GA_audience_2022.csv')
df_prec = pd.read_csv('../data/Weather_prec_2022.csv')

In [13]:
# Estandarizamos el dataframe para unirlo luego al df de estudio
# ===============================================================================

# Creamos un diccionario con los meses que queremos reemplazar
MONTH_NAMES = {
    202201: '2022-01-01',
    202202: '2022-02-01',
    202203: '2022-03-01',
    202204: '2022-04-01',
    202205: '2022-05-01',
    202206: '2022-06-01',
    202207: '2022-07-01',
    202208: '2022-08-01',
    202209: '2022-09-01',
    202210: '2022-10-01',
    202211: '2022-11-01',
    202212: '2022-12-01',
}

# Reemplazamos los números con el nombre del diccionario en la columna
df_GA['ga:yearMonth'] = df_GA['ga:yearMonth'].replace(MONTH_NAMES)

# Renombramos la columna para unirla al dataframe original
df_GA = df_GA.rename(columns={'ga:yearMonth': 'date','ga:channelGrouping':'channelGrouping',
                              'ga:sessions':'sessions','ga:avgSessionDuration':'avgSessionDuration',
                              'ga:users':'users'})

# Filtramos por 'Organic Search', que es la parte que nos interesa mirar, 
# ya que supone más del 98% del tráfico de nuestra web
df_GA = df_GA[df_GA['channelGrouping'] == 'Organic Search']

# Eliminamos la columna 'Unnamed: 0', que no necesitamos
df_GA = df_GA.drop(['Unnamed: 0','ga:newUsers','channelGrouping'], axis=1)

In [14]:
# Estandarizamos el dataframe para unirlo luego al df de estudio
# ===============================================================================

# Creamos un diccionario con los meses que queremos reemplazar
MONTH_NAMES = {
    1: '2022-01-01',
    2: '2022-02-01',
    3: '2022-03-01',
    4: '2022-04-01',
    5: '2022-05-01',
    6: '2022-06-01',
    7: '2022-07-01',
    8: '2022-08-01',
    9: '2022-09-01',
    10: '2022-10-01',
    11: '2022-11-01',
    12: '2022-12-01',
}

# Reemplazamos los números con el nombre del diccionario en la columna
df_prec['mes'] = df_prec['mes'].replace(MONTH_NAMES)

# Renombramos la columna 'mes' a 'date' y 'media' a 'media_prec'
df_prec = df_prec.rename(columns={'mes': 'date', 'media': 'media_prec'})

# Eliminamos la columna 'Unnamed: 0', que no necesitamos
df_prec = df_prec.drop('Unnamed: 0', axis=1)

In [15]:
# Mergeamos los df por la columna date
# ===============================================================================

df_merged = pd.merge(df_prec, df_GA, on='date', how='inner')

df_merged = pd.merge(df, df_merged, on='date', how='inner')

In [16]:
# Generamos una función para formatear los meses 
# ===============================================================================

def get_month_name(date_str):
    date_obj = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    return date_obj.date() #.strftime('%B')

df_merged['date'] = df_merged['date'].apply(get_month_name)

In [17]:
df_merged.describe().T

,count,mean,std,min,25%,50%,75%,max
revenue_resold,11732.0,3.747825e+01,3.607187e+02,8.000000e-06,1.158033e-02,1.613450e-01,2.977034e+00,3.031308e+04
impressions_resold,11732.0,2.791559e+04,1.709116e+05,1.000000e+00,8.000000e+00,1.050000e+02,2.078750e+03,7.090940e+06
cpm,11732.0,2.212293e+00,2.622359e+00,1.000000e-02,6.500000e-01,1.410000e+00,3.000000e+00,3.939000e+01
media_prec,11732.0,4.589341e+01,3.290722e+01,6.897170e+00,1.856226e+01,4.259434e+01,7.444717e+01,1.042200e+02
sessions,11732.0,3.740929e+07,9.913904e+06,2.460338e+07,2.930719e+07,3.607373e+07,4.037707e+07,5.874823e+07
avgSessionDuration,11732.0,1.153766e+02,5.951639e+00,1.047322e+02,1.113505e+02,1.177875e+02,1.209459e+02,1.242691e+02
users,11732.0,7.959504e+06,1.557506e+06,6.014006e+06,6.909242e+06,7.324029e+06,8.916953e+06,1.102696e+07


In [18]:
df_merged.describe(include="object").T

,count,unique,top,freq
date,11732,12,2022-04-01,1063
transaction_type_name,11732,3,Open auction,9305
brand_id_name,11732,22,Volkswagen Group - Volkswagen,993
format_name,11732,12,Mobile Banner,2774
device_name,9911,4,Mobile,4061
media_type,11732,3,Web,8559
industria,11732,34,AUTOS & VEHICLES : Vehicles - National,2354


In [19]:
# Hacemos una copia del df para continuar con el análsis
# ===============================================================================

df_merged2 = df_merged.copy()

In [20]:
# Creamos una función para randomizar los nombres de los clientes
# ===============================================================================

def random_string(length):
    """
    Función para generar una cadena aleatoria de letras mayúsculas y dígitos
    """
    letters_digits = string.ascii_uppercase + string.digits
    return ''.join(random.choice(letters_digits) for i in range(length))

# Genera un conjunto de nombres aleatorios únicos igual al número de valores únicos de la columna
unique_names = set(random_string(7) for i in range(len(df_merged2['brand_id_name'].unique())))

# Crea un diccionario asignando cada valor único a un nombre aleatorio
name_map = dict(zip(df_merged2['brand_id_name'].unique(), unique_names))

# Usa apply() y lambda para reemplazar cada valor con su correspondiente nombre aleatorio
df_merged2['brand_id_name'] = df_merged2['brand_id_name'].apply(lambda x: name_map[x])

df_merged2.to_csv('../data/merged_df_2022.csv')

In [21]:
# Mejores formatos
# ==============================================================================

df_merged2_mf= df_merged2.groupby(['format_name','date'],dropna=False).agg({'revenue_resold': 'sum'}).sort_values('revenue_resold', ascending=False).round(2).reset_index()

# Guardamos el resultado en un csv para unirlo a nuestro df original
df_merged2_mf.to_csv('../data/merged_df_2022_formatos.csv')

In [22]:
# Mejores Transaction type
# ==============================================================================

df_merged2_tp= df_merged2.groupby(['transaction_type_name','date'],dropna=False).agg({'revenue_resold': 'sum'}).sort_values('revenue_resold', ascending=False).round(2).reset_index()

# Guardamos el resultado en un csv para unirlo a nuestro df original
df_merged2_tp.to_csv('../data/merged_df_2022_transaction_type.csv')

In [23]:
# Mejores brand_id_name
# ==============================================================================

df_merged2_bd= df_merged2.groupby(['brand_id_name','date'],dropna=False).agg({'revenue_resold': 'sum'}).sort_values('revenue_resold', ascending=False).round(2).reset_index()

# Guardamos el resultado en un csv para unirlo a nuestro df original
df_merged2_bd.to_csv('../data/merged_df_2022_brand_name.csv')

In [24]:
# Mejores device_name
# ==============================================================================

df_merged2_dn= df_merged2.groupby(['device_name','date'],dropna=False).agg({'revenue_resold': 'sum'}).sort_values('revenue_resold', ascending=False).round(2).reset_index()

# Guardamos el resultado en un csv para unirlo a nuestro df original
df_merged2_dn.to_csv('../data/merged_df_2022_device_name.csv')

In [25]:
# Mejores media type
# ==============================================================================

df_merged2_mt= df_merged2.groupby(['media_type','date'],dropna=False).agg({'revenue_resold': 'sum'}).sort_values('revenue_resold', ascending=False).round(2).reset_index()

# Guardamos el resultado en un csv para unirlo a nuestro df original
df_merged2_mt.to_csv('../data/merged_df_2022_media_type.csv')

In [26]:
# Mejores industria
# ==============================================================================

df_merged2_ind= df_merged2.groupby(['industria','date'],dropna=False).agg({'revenue_resold': 'sum'}).sort_values('revenue_resold', ascending=False).round(2).reset_index()

# Guardamos el resultado en un csv para unirlo a nuestro df original
df_merged2_ind.to_csv('../data/merged_df_2022_ind.csv')

In [27]:
# Mejores date
# ==============================================================================

df_merged2_date= df_merged.groupby(['date','media_prec','sessions','users'],dropna=False).agg({'revenue_resold': 'sum','cpm':'mean'}).sort_values('revenue_resold', ascending=False).round(2).reset_index()

df_merged2_date['percentage'] = (df_merged2_date['revenue_resold']/df_merged2_date['revenue_resold'].sum()*100).round(2)

# Guardamos el resultado en un csv para unirlo a nuestro df original
df_merged2_date.to_csv('../data/merged_df_2022_date.csv')